<a href="https://colab.research.google.com/github/The-0mnipotent/ML-Project-Grp1-Titanic/blob/main/Titanic_Problem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np

a = pd.read_csv('train.csv')
b = pd.read_csv('test.csv')

e = pd.DataFrame({'PassengerId': b.PassengerId})

a.head()

FileNotFoundError: ignored

In [3]:
print(a['Cabin'].isnull().sum())
print(a['Cabin'].nunique())

NameError: ignored

In [ ]:
a = a.drop(['PassengerId', 'Name', 'Ticket'], axis=1)
b = b.drop(['PassengerId', 'Name', 'Ticket'], axis=1)

a.head()

In [ ]:
a.hist(column='Fare', bins=20)

In [ ]:

def f0(row):
  if row['Fare']<=90:
    value=0
  else:
    value=1
  return value

a['Fare'] = a.apply(f0, axis=1)

b['Fare'] = b.apply(f0, axis=1)


In [ ]:
a['Cabin'] = pd.factorize(a['Cabin'])[0]
a['Sex'] = pd.factorize(a['Sex'])[0]

b['Cabin'] = pd.factorize(b['Cabin'])[0]
b['Sex'] = pd.factorize(b['Sex'])[0]

def f(row):
  if row['Cabin']==-1:
    value=0
  else:
    value=1
  return value

a['Cabin_Exists'] = a.apply(f, axis=1)
b['Cabin_Exists'] = b.apply(f, axis=1)

a.head()

In [ ]:
a = a.drop(['Cabin'], axis=1)
b = b.drop(['Cabin'], axis=1)
a.corr()

In [ ]:
def f1(row):
  value = row['SibSp'] + row['Parch']
  return value

a['Family'] = a.apply(f1, axis=1)
b['Family'] = b.apply(f1, axis=1)

a = a.drop(['SibSp', 'Parch'], axis=1)
b = b.drop(['SibSp', 'Parch'], axis=1)

a.head()

In [ ]:
from sklearn.ensemble import RandomForestRegressor

# split data
train = a.loc[(a['Age'].notnull())]  # known age values
test = a.loc[(a['Age'].isnull())]  # all nan age values

train_b = b.loc[(b['Age'].notnull())]  # known age values
test_b = b.loc[(b['Age'].isnull())]  # all nan age values

# select age column
y = train.values[:, 3]
y_b = train_b.values[:, 2]

# select pclass, fare and family
X = train.values[:, [1, 7]]
X_b = train_b.values[:, [0, 6]]

# create RandomForestRegressor model
rfr = RandomForestRegressor(n_estimators=100, n_jobs=-1, random_state=7)

# Fit a model
rfr.fit(X, y)
rfr.fit(X_b, y_b)

# Use the fitted model to predict the missing values
predictedAges = rfr.predict(test.values[:, [1, 7]])

predictedAges_b = rfr.predict(test_b.values[:, [0, 6]])

# create predicted age column
a['pred_age'] = a['Age']

b['pred_age'] = b['Age']


# fill column
a.loc[(a['pred_age'].isnull()), 'pred_age'] = predictedAges 

b.loc[(b['pred_age'].isnull()), 'pred_age'] = predictedAges_b 


a = a.drop(['Age'], axis=1)

b = b.drop(['Age'], axis=1)

a.head()

In [ ]:
a['Embarked'] = a['Embarked'].fillna(a['Embarked'].mode()[0])
a['Embarked'] = pd.factorize(a['Embarked'])[0]

b['Embarked'] = b['Embarked'].fillna(b['Embarked'].mode()[0])
b['Embarked'] = pd.factorize(b['Embarked'])[0]

# a = a.drop('Embarked', axis=1)
# b = b.drop('Embarked', axis=1)


In [ ]:
a.isnull().sum()

In [ ]:
a.corr()

In [ ]:
a.hist(column='pred_age', bins=20)


In [ ]:
a.groupby('pred_age')['Survived'].mean().plot(kind='bar', figsize=(25,9))


In [ ]:
def f2(row):
  if row['pred_age']<=13:
    value=0#infant
  elif row['pred_age']<=60:
    value=1#young
  else:
    value=2 #old
  return value

a['pred_age'] = a.apply(f2, axis=1)

b['pred_age'] = b.apply(f2, axis=1)

a.head()

In [ ]:
c = a['Survived']
a = a.drop(['Survived'], axis=1)

c.head()

In [ ]:

from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(a,c,test_size=0.3,random_state=7)

from sklearn.preprocessing import StandardScaler

scaler=StandardScaler()

scaler.fit(X_train)

scaled_X_train=scaler.transform(X_train)
scaled_X_test=scaler.transform(X_test)


from sklearn.neighbors import KNeighborsClassifier

#Create KNN Classifier
knn = KNeighborsClassifier(n_neighbors=3)


from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

from sklearn.model_selection import GridSearchCV


kf = KFold(n_splits=10, shuffle=True, random_state=7)


In [ ]:
# knn_params = {'n_neighbors':list(range(3,11))}

# GS1 = GridSearchCV(knn, knn_params, cv = kf, scoring='accuracy')

# GS1.fit(scaled_X_train, y_train)

# print(GS1.best_params_)

# print(GS1.best_score_)

In [ ]:

# kf = KFold(n_splits=10, shuffle=True, random_state=7)
 
# result = cross_val_score(knn , scaled_X_train, y_train, cv = kf)
 
# print(result.mean())


In [ ]:
from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression()

result = cross_val_score(logreg , scaled_X_train, y_train, cv = kf)
 
print(result.mean())
 

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

gb_clf = GradientBoostingClassifier(n_estimators=120, learning_rate=0.3, max_depth=9, random_state=7)

# GB_params = {'n_estimators': [20, 60, 120, 180], 'learning_rate': [0.05, 0.1, 0.2, 0.3, 0.4, 0.5], 'max_depth': list(range(5, 12))}

# GS3 = GridSearchCV(gb_clf, GB_params, cv = kf, scoring='accuracy')

# GS3.fit(scaled_X_train, y_train)

# print(GS3.best_params_)

# print(GS3.best_score_) 



In [ ]:
from sklearn.ensemble import RandomForestClassifier

clfRF = RandomForestClassifier(n_estimators=100, criterion='entropy', max_depth=9, bootstrap=True, n_jobs=-1, random_state=7)


In [ ]:

# kf = KFold(n_splits=10, shuffle=True, random_state=7)


# RF_params = {'n_estimators': [100, 300, 500, 800, 1000], 'criterion': ['gini', 'entropy'], 'max_depth': list(range(5, 12))}

# GS2 = GridSearchCV(clfRF, RF_params, cv = kf, scoring='accuracy')

# GS2.fit(scaled_X_train, y_train)

# print(GS2.best_params_)

# print(GS2.best_score_)



In [ ]:
from sklearn.neural_network import MLPClassifier

mlp = MLPClassifier(hidden_layer_sizes=(8,10,8), activation='relu', solver='adam', max_iter=500)

#cross validation
 
result = cross_val_score(mlp , scaled_X_train, y_train, cv = kf)
 
print(result.mean())

In [ ]:
from sklearn.ensemble import VotingClassifier


clf = VotingClassifier(estimators = [('knn', knn), ('GB', gb_clf), ('RFC', clfRF), ('LR', logreg), ('MLP', mlp)], voting = 'hard')

clf = clf.fit(scaled_X_train, y_train)

y_pred = clf.predict(scaled_X_test)

#cross-validation
 
result = cross_val_score(clf , scaled_X_train, y_train, cv = kf)
 
print(result.mean())

In [ ]:


from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

print(confusion_matrix(y_test, y_pred))
print(accuracy_score(y_test, y_pred))

print(classification_report(y_test, y_pred))

In [ ]:
# test_b = scaler.transform(b)
# test_pred = clf.predict(test_b)

# output = pd.DataFrame({'PassengerId': e.PassengerId, 'Survived': test_pred})
# print(output)
# output.to_csv('submission.csv', index=False)

# print("Submission saved")